In [1]:
import tflearn
import numpy as np
from tflearn.layers.normalization import batch_normalization as batchnorm
from sklearn.preprocessing import normalize
data_train = np.load('data_train.npy')
data_train = np.random.permutation(data_train)
X_train = normalize(data_train[:,:len(data_train[0])-2])
Y_train = data_train[:, len(data_train[0])-2:]

In [2]:
width0 = len(X_train[0])
width1 = int(2*width0)
width2 = int(width0/2)

net = tflearn.input_data(shape=[None, width0])
net = tflearn.fully_connected(batchnorm(net), width1, activation='relu')
net = tflearn.fully_connected(batchnorm(net), width2, activation='relu')
net = tflearn.fully_connected(net, 2, activation='softmax')
net    = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(X_train, Y_train, n_epoch=10, batch_size=64, show_metric=True)

Training Step: 62499  | total loss: 0.09047 | time: 30.438s
| Adam | epoch: 010 | loss: 0.09047 - acc: 0.9715 -- iter: 399936/400000
Training Step: 62500  | total loss: 0.09382 | time: 30.443s
| Adam | epoch: 010 | loss: 0.09382 - acc: 0.9681 -- iter: 400000/400000
--


In [53]:
data_test = np.load('data_test.npy')
X_test = normalize(data_test[:,:len(data_test[0])-2])
Y_test = data_test[:, len(data_test[0])-2:]


print(model.evaluate(X_train, Y_train))
print(model.evaluate(X_test, Y_test))

[0.97984749999999998]
[0.97757249999999996]


In [ ]:
y_train = []
for i in range(len(Y_train)):
    y_train += [Y_train[i][1]]

from sklearn.linear_model import LogisticRegression
skmodel = LogisticRegression()
skmodel.fit(X_train, y_train)
print(skmodel.score(X_train, y_train))

In [52]:
#!/usr/bin/env python -W ignore::DeprecationWarning
f = open("vec.emd","r")
x=f.readline()
dim=int(x.split()[1])
numNodes=int(x.split()[0])

node2vec = {}
for _ in range(numNodes):
    t=f.readline().split()
    node2vec[int(t[0])]=np.array(list(map(float,t[1:])))
    
import pickle
f = open('neighbor_data.pkl', 'rb')
l = pickle.load(f, encoding='latin1')
from scipy.spatial.distance import cosine
import numpy as np
it = 0
results = []
lengths = []
for x in l:
    myvec, truei, truevec, lshi, lshvec = x
    lshi = list(lshi)
    lshvec = [node2vec[i] for i in lshi]
    candidates = []
    for i in range(len(lshi)):
        cos = cosine(myvec, lshvec[i])
        diff = np.array(myvec)-np.array(lshvec[i])
        X = normalize(np.append(diff, cos).reshape(-1,61))[0]
        pred = model.predict_label(np.array([X]))
        if pred[0][0] == 0:
            candidates += [list(lshi)[i]]
    count = 0.0
    for y in candidates:
        if y in truei:
            count+=1
    if len(candidates)>0:
        results+=[count/len(candidates)]
        lengths += [count/len(truei)]
print(np.mean(results))
print(np.mean(lengths))

0.307331809134
0.427821009062
